In [1]:
import numpy as np
import genesis as gs

[I 03/18/25 14:43:11.751 99373] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:
gs.init(backend=gs.cpu)

[Genesis] [14:43:14] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [14:43:14] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [14:43:14] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [14:43:14] [INFO] Running on [11th Gen Intel(R) Core(TM) i9-11900K @ 3.50GHz] with backend gs.cpu. Device memory: 62.64 GB.
[Genesis] [14:43:15] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.


In [3]:
scene = gs.Scene(
    show_viewer    = True,
    viewer_options = gs.options.ViewerOptions(
        res           = (1280, 960),
        camera_pos    = (3.5, 0.0, 2.5),
        camera_lookat = (0.0, 0.0, 0.5),
        camera_fov    = 30,
        max_FPS       = 60,
    ),
    vis_options = gs.options.VisOptions(
        show_world_frame = True, # visualize the coordinate frame of `world` at its origin
        world_frame_size = 1.0, # length of the world frame in meter
        show_link_frame  = False, # do not visualize coordinate frames of entity links
        show_cameras     = False, # do not visualize mesh and frustum of the cameras added
        plane_reflection = True, # turn on plane reflection
        ambient_light    = (0.1, 0.1, 0.1), # ambient light setting
    ),
    sim_options = gs.options.SimOptions(
        dt = 0.01,
        substeps = 4,
    ),
    renderer = gs.renderers.Rasterizer(), # using rasterizer for camera rendering
)

[Genesis] [14:43:17] [INFO] Scene <ad42912> created.


In [4]:
plane = scene.add_entity(gs.morphs.Plane())

panda = scene.add_entity(
    gs.morphs.MJCF(file='mujoco_menagerie/franka_emika_panda/panda.xml',)
)

cube = scene.add_entity(
    gs.morphs.Box(
        size = (0.04, 0.04, 0.08),
        pos = (0.65, 0.0, 0.25),
    ),
    surface=gs.surfaces.Default(
        color=(1, 0.8, 0),
    )
)

[Genesis] [14:43:18] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <dbea6e3>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [14:43:18] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <21e7210>, morph: <gs.morphs.MJCF(file='/home/a3r/Interns/internship2024-25/genesis/mujoco_menagerie/franka_emika_panda/panda.xml')>, material: <gs.materials.Rigid>.
[Genesis] [14:43:20] [INFO] Adding <gs.RigidEntity>. idx: 2, uid: <b6c483a>, morph: <gs.morphs.Box>, material: <gs.materials.Rigid>.


In [5]:
cam = scene.add_camera(
    res = (640, 480),
    pos = (0.45, -2.5, 1.25),
    lookat = (0.65, 0, 0.25),
    fov = 20,
    GUI = True,
)

camFilm = scene.add_camera(
    res = (640, 480),
    pos = (0.45, -2.5, 1.25),
    lookat = (0.65, 0, 0.25),
    fov = 20,
    GUI = True,
)


In [6]:
scene.build()
camFilm.start_recording()

[Genesis] [14:43:22] [INFO] Building scene <ad42912>...
[W 03/18/25 14:43:23.046 99373] [frontend_ir.cpp:begin_frontend_struct_for_on_snode@1678] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.
[W 03/18/25 14:43:23.377 99373] [frontend_ir.cpp:begin_frontend_struct_for_on_snode@1678] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.
[W 03/18/25 14:43:26.049 99373] [frontend_ir.cpp:begin_frontend_struct_for_on_snode@1678] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.
[Genesis] [14:43:26] [INFO] Compiling simulation kernels...
[Genesis] [14:43:29] [INFO] Building visualizer...
[Genesis] [14:43:33] [INFO] Viewer created. Resolution: 1280×960, max_FPS: 60.


In [7]:
motors_dof = np.arange(7)
fingers_dof = np.arange(7,9)

In [8]:
# these are tuned for the specific robot (panda)
panda.set_dofs_kp(
    np.array([4500, 4500, 3500, 3500, 2000, 2000, 2000, 100, 100]),
)
panda.set_dofs_kv(
    np.array([450, 450, 350, 350, 200, 200, 200, 10, 10]),
)
panda.set_dofs_force_range(
    np.array([-87, -87, -87, -87, -12, -12, -12, -100, -100]),
    np.array([87, 87, 87, 87, 12, 12, 12, 100, 100]),
)

In [9]:
# get the end-effector link
end_effector = panda.get_link('hand')

In [10]:
# move to pose before grasp
qpos = panda.inverse_kinematics(
    link = end_effector,
    pos = np.array([0.65, 0.0, 0.25]),
    quat = np.array([0, 1, 0, 0]),
)

In [11]:
panda.control_dofs_position(qpos[:-2], motors_dof)

In [12]:
for i in range(100):
    scene.step()

[Genesis] [14:43:44] [INFO] Running at 1.86 FPS.
[Genesis] [14:43:44] [INFO] Running at 1.95 FPS.
[Genesis] [14:43:44] [INFO] Running at 2.05 FPS.
[Genesis] [14:43:44] [INFO] Running at 2.15 FPS.
[Genesis] [14:43:44] [INFO] Running at 2.26 FPS.
[Genesis] [14:43:44] [INFO] Running at 2.38 FPS.
[Genesis] [14:43:44] [INFO] Running at 2.50 FPS.
[Genesis] [14:43:44] [INFO] Running at 2.62 FPS.
[Genesis] [14:43:44] [INFO] Running at 2.76 FPS.
[Genesis] [14:43:44] [INFO] Running at 2.89 FPS.
[Genesis] [14:43:44] [INFO] Running at 3.04 FPS.
[Genesis] [14:43:44] [INFO] Running at 3.19 FPS.
[Genesis] [14:43:44] [INFO] Running at 3.35 FPS.
[Genesis] [14:43:44] [INFO] Running at 3.51 FPS.
[Genesis] [14:43:44] [INFO] Running at 3.69 FPS.
[Genesis] [14:43:44] [INFO] Running at 3.87 FPS.
[Genesis] [14:43:44] [INFO] Running at 4.06 FPS.
[Genesis] [14:43:44] [INFO] Running at 4.26 FPS.
[Genesis] [14:43:44] [INFO] Running at 4.47 FPS.
[Genesis] [14:43:44] [INFO] Running at 4.68 FPS.
[Genesis] [14:43:44]

In [13]:
currentPos = end_effector.get_pos()
print(currentPos)

tensor([ 6.4830e-01, -4.6931e-04,  2.4198e-01])


In [14]:
import cv2
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor
from PIL import Image

/home/a3r/miniconda3/envs/openvla/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-18 14:43:49.845048: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-18 14:43:49.845071: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-18 14:43:49.845727: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-18 14:43:50.373973: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [19]:
processor = AutoProcessor.from_pretrained("openvla/openvla-7b", trust_remote_code=True)
vla = AutoModelForVision2Seq.from_pretrained(
    "openvla/openvla-7b", 
    #attn_implementation="flash_attention_2",  # [Optional] Requires `flash_attn`
    torch_dtype=torch.bfloat16, 
    low_cpu_mem_usage=True, 
    trust_remote_code=True
).to("cuda:0")

ImportError: This modeling file requires the following packages that were not found in your environment: vla. Run `pip install vla`

In [16]:
prompt = "In: What action should the robot take to touch the yellow block?\nOut:"


In [1]:


cam.set_pose(pos=(1.25, -2.5, 1.3), lookat=(1.25, 0, 0.25))

cam.render()

NameError: name 'cam' is not defined

In [2]:
camFilm.set_pose(pos=(2.5, -2.5, 1.65), lookat=(0.65, 0, 0.25))

camFilm.render()


NameError: name 'camFilm' is not defined

In [19]:
for i in range(100):
    cam.start_recording()
    for i in range(25):
        scene.step()
        cam.render()
        camFilm.render()

    cam.stop_recording(save_to_filename='clip.mp4')

    currentPos = end_effector.get_pos()
    print(currentPos)

    video = cv2.VideoCapture('clip.mp4')
    video.set(cv2.CAP_PROP_POS_FRAMES, 9)
    ret, frame = video.read()
    print(ret)
    cv2.imwrite('frame.jpg', frame)
    image = Image.open('frame.jpg')

    inputs = processor(prompt, image).to("cuda:0", dtype=torch.bfloat16)
    action = vla.predict_action(**inputs, unnorm_key="bridge_orig", do_sample=False)

    qpos= panda.inverse_kinematics(
        link = end_effector,
        pos = np.array([currentPos[0]-action[1], currentPos[1]-action[0], currentPos[2]+action[2]]),
    )

    panda.control_dofs_position(qpos[:-2], motors_dof)

[Genesis] [18:07:43] [INFO] Running at 3.10 FPS.


[Genesis] [18:07:43] [INFO] Running at 3.20 FPS.
[Genesis] [18:07:43] [INFO] Running at 3.35 FPS.
[Genesis] [18:07:43] [INFO] Running at 3.51 FPS.
[Genesis] [18:07:43] [INFO] Running at 3.69 FPS.
[Genesis] [18:07:43] [INFO] Running at 3.87 FPS.
[Genesis] [18:07:43] [INFO] Running at 4.04 FPS.
[Genesis] [18:07:43] [INFO] Running at 4.23 FPS.
[Genesis] [18:07:43] [INFO] Running at 4.44 FPS.
[Genesis] [18:07:43] [INFO] Running at 4.62 FPS.
[Genesis] [18:07:43] [INFO] Running at 4.84 FPS.
[Genesis] [18:07:43] [INFO] Running at 5.08 FPS.
[Genesis] [18:07:43] [INFO] Running at 5.28 FPS.
[Genesis] [18:07:43] [INFO] Running at 5.51 FPS.
[Genesis] [18:07:43] [INFO] Running at 5.76 FPS.
[Genesis] [18:07:43] [INFO] Running at 5.99 FPS.
[Genesis] [18:07:43] [INFO] Running at 6.26 FPS.
[Genesis] [18:07:43] [INFO] Running at 6.53 FPS.
[Genesis] [18:07:43] [INFO] Running at 6.81 FPS.
[Genesis] [18:07:43] [INFO] Running at 7.07 FPS.
[Genesis] [18:07:44] [INFO] Running at 7.38 FPS.
[Genesis] [18:07:44]

tensor([ 6.4834e-01, -4.6508e-04,  2.4258e-01])
True


[Genesis] [18:07:44] [INFO] Running at 6.93 FPS.
[Genesis] [18:07:44] [INFO] Running at 7.25 FPS.
[Genesis] [18:07:44] [INFO] Running at 7.58 FPS.
[Genesis] [18:07:44] [INFO] Running at 7.93 FPS.
[Genesis] [18:07:44] [INFO] Running at 8.23 FPS.
[Genesis] [18:07:44] [INFO] Running at 8.58 FPS.
[Genesis] [18:07:44] [INFO] Running at 8.86 FPS.
[Genesis] [18:07:44] [INFO] Running at 9.23 FPS.
[Genesis] [18:07:45] [INFO] Running at 9.64 FPS.
[Genesis] [18:07:45] [INFO] Running at 10.06 FPS.
[Genesis] [18:07:45] [INFO] Running at 10.36 FPS.
[Genesis] [18:07:45] [INFO] Running at 10.71 FPS.
[Genesis] [18:07:45] [INFO] Running at 11.13 FPS.
[Genesis] [18:07:45] [INFO] Running at 11.60 FPS.
[Genesis] [18:07:45] [INFO] Running at 11.90 FPS.
[Genesis] [18:07:45] [INFO] Running at 12.36 FPS.
[Genesis] [18:07:45] [INFO] Running at 12.66 FPS.
[Genesis] [18:07:45] [INFO] Running at 13.12 FPS.
[Genesis] [18:07:45] [INFO] Running at 13.66 FPS.
[Genesis] [18:07:45] [INFO] Running at 13.95 FPS.
[Genesis]

tensor([0.6721, 0.0023, 0.2300])
True


...

tensor([0.7090, 0.0534, 0.1118])
True


[Genesis] [18:09:28] [INFO] Running at 17.71 FPS.
[Genesis] [18:09:28] [INFO] Running at 18.35 FPS.
[Genesis] [18:09:28] [INFO] Running at 18.73 FPS.
[Genesis] [18:09:28] [INFO] Running at 19.09 FPS.
[Genesis] [18:09:28] [INFO] Running at 19.66 FPS.
[Genesis] [18:09:28] [INFO] Running at 19.80 FPS.
[Genesis] [18:09:28] [INFO] Running at 20.32 FPS.
[Genesis] [18:09:28] [INFO] Running at 20.47 FPS.
[Genesis] [18:09:28] [INFO] Running at 21.02 FPS.
[Genesis] [18:09:28] [INFO] Running at 21.12 FPS.
[Genesis] [18:09:28] [INFO] Running at 21.73 FPS.
[Genesis] [18:09:28] [INFO] Running at 22.40 FPS.
[Genesis] [18:09:28] [INFO] Running at 22.49 FPS.
[Genesis] [18:09:28] [INFO] Running at 23.05 FPS.
[Genesis] [18:09:28] [INFO] Running at 23.75 FPS.
[Genesis] [18:09:28] [INFO] Running at 23.76 FPS.
[Genesis] [18:09:28] [INFO] Running at 24.32 FPS.
[Genesis] [18:09:28] [INFO] Running at 25.01 FPS.
[Genesis] [18:09:28] [INFO] Running at 25.78 FPS.
[Genesis] [18:09:28] [INFO] Running at 25.73 FPS.


tensor([0.7102, 0.0539, 0.1120])
True


[Genesis] [18:09:29] [INFO] Running at 17.85 FPS.
[Genesis] [18:09:29] [INFO] Running at 18.28 FPS.
[Genesis] [18:09:29] [INFO] Running at 18.83 FPS.
[Genesis] [18:09:29] [INFO] Running at 19.50 FPS.
[Genesis] [18:09:29] [INFO] Running at 19.65 FPS.
[Genesis] [18:09:29] [INFO] Running at 20.20 FPS.
[Genesis] [18:09:29] [INFO] Running at 20.36 FPS.
[Genesis] [18:09:29] [INFO] Running at 20.94 FPS.
[Genesis] [18:09:29] [INFO] Running at 21.64 FPS.
[Genesis] [18:09:29] [INFO] Running at 21.71 FPS.
[Genesis] [18:09:29] [INFO] Running at 22.30 FPS.
[Genesis] [18:09:29] [INFO] Running at 23.03 FPS.
[Genesis] [18:09:29] [INFO] Running at 22.98 FPS.
[Genesis] [18:09:29] [INFO] Running at 23.66 FPS.
[Genesis] [18:09:29] [INFO] Running at 23.62 FPS.
[Genesis] [18:09:29] [INFO] Running at 24.18 FPS.
[Genesis] [18:09:29] [INFO] Running at 24.93 FPS.
[Genesis] [18:09:29] [INFO] Running at 25.35 FPS.
[Genesis] [18:09:29] [INFO] Running at 25.90 FPS.
[Genesis] [18:09:29] [INFO] Running at 26.67 FPS.


tensor([0.7089, 0.0548, 0.1120])
True


In [20]:
camFilm.stop_recording(save_to_filename='film.mp4')

[Genesis] [18:09:30] [INFO] Saving video to film.mp4.
[Genesis] [18:09:38] [INFO] Video saved.
